In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
proj_path = "/content/drive/MyDrive/2023/2023_하계_시그널하우스_연구체험 인턴십/01_BIS prediction using EEG signals"

# 전처리 1개

In [ ]:
data0 = np.load(f'{proj_path}/data/raw/0.npy')

print(data0)

In [ ]:
print('shape : ',data0.shape)
print('size : ', data0.size)

In [ ]:
np.isnan(data0)[np.isnan(data0) == False].size

In [ ]:
#BIS nan아닌 애들 indexing

bis = data0[:, 0]
bis_cond = ~np.isnan(bis)

In [ ]:
bis_cond

In [ ]:
idx_arr = np.arange(0, len(bis), 1)
bis_idx = idx_arr[bis_cond]

In [ ]:
bis_idx.shape

In [ ]:
bis_idx

In [ ]:
#SQI nan 아닌 애들 indexing

sqi = data0[:,1]
sqi_cond = ~np.isnan(sqi)

In [ ]:
np.isnan(sqi)[np.isnan(sqi) == False].size

In [ ]:
idx_arr_s = np.arange(0, len(sqi), 1)
sqi_idx = idx_arr_s[sqi_cond]

In [ ]:
sqi_idx

In [ ]:
#EEG 30s씩 자르기

start = 55*100

eeg_start_idx = bis_idx - start
eeg_start_idx = eeg_start_idx[eeg_start_idx >= 0]

In [ ]:
eeg = data0[:, 2]
eeg

In [ ]:
eeg.shape

In [ ]:
a = np.isnan(bis)[np.isnan(bis) == True].size
b = np.isnan(sqi)[np.isnan(sqi) == True].size
c = np.isnan(eeg)[np.isnan(eeg) == True].size

In [ ]:
print( "BIS nan 값 개수 : " , a)
print("SQI nan 값 개수 : " , b )
print("EEG nan 값 개수 : " , c)

In [ ]:
eeg_start_idx.shape

In [ ]:
eeg_start_idx

In [ ]:
from tqdm import tqdm

In [ ]:
eeg_segments = []
bis_label = []
sqi_label = []
for start_idx in tqdm(eeg_start_idx):
  end_idx = start_idx + 30 * 100
  eeg_segments += list([eeg[start_idx:end_idx]])
  bis_label += list([bis[start_idx]])
  sqi_label += list([sqi[start_idx]])

In [ ]:
eeg_segments = np.array(eeg_segments)
bis_label = np.array(bis_label)
sqi_label = np.array(sqi_label)

In [ ]:
eeg_segments

In [ ]:
bis_label.shape

In [ ]:
sqi_label

In [ ]:
# SQI > 90인 condition 만들기
sqi_cond2 = sqi_label > 90
sqi_cond2

In [ ]:
# # BIS, SQI, EEG 모두 SQI > 90으로 filtering

# bis_label = bis_label[sqi_cond2]
# sqi_label = sqi_label[sqi_cond2]
# eeg_selected = eeg_segments[sqi_cond2]

In [ ]:
#directory 생성해서 전처리 완료한 결과물을 저장

In [ ]:
bis_label.shape, sqi_label.shape, eeg_selected.shape

In [ ]:
#eeg flatten 1차원 array 생성
eeg_flat = eeg_selected.flatten()

# EEG 결측값 정리

#EEG 이상치 제거하기

In [ ]:
from scipy.stats import mode

In [ ]:
eeg

In [ ]:
# # 대표값
# # print('평균 : ', np.mean(eeg_segments))
# # print('중앙값 : ', np.median(data))
# # print('최빈값 : ', np.bincount(data).argmax())
# print('제 1 사분위수 : ', np.quantile(eeg, 0.25))
# print('제 2 사분위수 : ', np.quantile(eeg, 0.5))
# print('제 3 사분위수 : ', np.quantile(eeg, 0.75))

# print('')
# # 최대 최소
# print('최대값 : ', np.max(eeg))
# print('최소값 : ', np.min(eeg))
# print('')
# # 산포를 나타내는 통계량 ddof=True 분모 자유도, ddof=False 분모 데이터 개수
# print('표본 표준편차(자유도) : ', np.std(eeg, ddof=True))
# print('표본 표준편차(데이터 개수) : ', np.std(eeg, ddof=False))
# print('표본 분산(자유도) : ', np.var(eeg, ddof=True))
# print('표본 분산(데이터 개수) : ', np.var(eeg, ddof=False))
print('IQR : ', np.subtract(*np.quantile(eeg, [0.75, 0.25]))) ## 또는 np.quantile(data, 0.75)-np.quantile(data, 0.25)
# print('범위 : ', np.max(eeg)-np.min(eeg))

In [ ]:
import seaborn as sns
sns.kdeplot(eeg, shade = True)
plt.show()

In [ ]:
Q3, Q1 = np.percentile(eeg, [75,25])
IQR = Q3 - Q1
IQR

In [ ]:
#이상치 제거

def outliers_iqr(data):
  q1, q3 = np.percentile(data, [25,75])
  iqr = q3 - q1
  lower_bound = q1 - (iqr * 1.5)
  upper_bound = q3 + (iqr * 1.5)

  return iqr
  # outlier = np.where((data > upper_bound)|(data < lower_bound))
  # return outlier

nan

In [ ]:
bis_label.shape

In [ ]:
test10 = np.array([7,5,6,6,5,7,4,6,3,4,5,7,8,3,6])
outliers_iqr(test10)

In [ ]:
q1 = np.percentile(eeg, 25)
q1

In [ ]:
eeg_segments_del = outliers_iqr(eeg_segments)

In [ ]:
eeg_segments_del

In [ ]:
eeg_del = outliers_iqr(eeg)[0]

In [ ]:
np.array(eeg_del).shape

In [ ]:
eeg_del

In [ ]:
#eeg_segments 정규분포

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

x = eeg
plt.figure(figsize=(15,10))
plt.title('Standard Normal Distribution')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid()
plt.plot(x, norm.pdf(x, loc=0, scale=1))
plt.show()

In [ ]:
import seaborn as sns
sns.boxplot(data=eeg)

In [ ]:
eeg_segments.shape, bis_label.shape

In [ ]:
#추출 test

test = [-9,-7,-5,-3,0,3,5,7,9]
test_abs = np.abs(test)
test_abs = np.array(test_abs)
test_abs

idx_test = np.arange(0, len(test), 1)
test_con = test_abs < 7
test_con
test_sel = test_abs[test_con]
test_sel

test_sel

In [ ]:
#eeg max 값 뽑아내기
# eeg_segments로부터 출발 (19575, 3000)

eeg_seg_abs = np.abs(eeg_segments)

map(max, eeg_seg_abs)
eeg_max_list = list(map(max, eeg_seg_abs))

eeg_max_list = np.array(eeg_max_list)
eeg_max_list


In [ ]:
# #test

# # eeg_seg_abs = np.abs(eeg_segments)

# eeg_seg = eeg_segments
# map(max, eeg_seg)
# eeg_max_list = list(map(max, eeg_seg))

# eeg_max_list = np.array(eeg_max_list)
# eeg_max_list

# eeg_delcon2 = np.abs(eeg_max_list) < 500
# eeg_delcon2

# eeg_maxsel2 = eeg_seg[eeg_delcon2]
# eeg_maxsel2.shape

In [ ]:
eeg_max_list.shape

In [ ]:
# np.set_printoptions(threshold=np.inf, linewidth=np.inf)
eeg_delcon = eeg_max_list < 500
eeg_delcon

In [ ]:
eeg_maxsel = eeg_seg_abs[eeg_delcon]

In [ ]:
# bis_label_del = np.arange(0, len(bis_label), 1)
bis_label_del = bis_label[eeg_delcon]

In [ ]:
# bis_label.shape,
eeg_maxsel.shape

(17799, 3000)

In [ ]:
bis_label.shape

In [ ]:
#시각화

x = np.linspace(0,17799, 17799)
b = eeg_maxsel
# plt.ticklabel_format(style='plain')

fig, ax = plt.subplots(figsize = (40,4))

plt.plot(x,b, color = 'navy')
plt.xlabel('index', fontsize =20)
plt.ylabel('EEG(μV)', fontsize = 20)
plt.title('delete outliers of EEG', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)


plt.ticklabel_format(style='plain')

# ax.set_xticks(minor_xticks, minor=True)
ax.yaxis.get_ticklocs(minor=True)
ax.minorticks_on()
ax.xaxis.get_ticklocs(minor=True)

ax.tick_params(axis='x', labelsize=20, length=10, width=3)
ax.tick_params(axis='x', which='minor', length=5, width=2)
# plt.fill_between(x, b, color = 'navy')


# 시각화

In [ ]:
#filtering 후 time- eeg 그래프 x축 time 맞다
#filtering 전은 EEG_01에 있음

x = np.linspace(0,10864,10864)
b = eeg_selected
# plt.ticklabel_format(style='plain')

fig, ax = plt.subplots(figsize = (40,4))

plt.plot(x,b, color = 'navy')
plt.xlabel('time(sec)', fontsize =20)
plt.ylabel('EEG(μV)', fontsize = 20)
plt.title('time-EEG', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)


plt.ticklabel_format(style='plain')

# ax.set_xticks(minor_xticks, minor=True)
ax.yaxis.get_ticklocs(minor=True)
ax.minorticks_on()
ax.xaxis.get_ticklocs(minor=True)

ax.tick_params(axis='x', labelsize=20, length=10, width=3)
ax.tick_params(axis='x', which='minor', length=5, width=2)
# plt.fill_between(x, b, color = 'navy')


In [ ]:
#filtering 후 time- bis 그래프

x = np.linspace(0,10864,10864)
b = bis_label
# minor_xticks = [i for i in range(0,400,10864)]

fig, ax = plt.subplots(figsize = (40,4))


plt.plot(x,b, color = 'navy')
plt.xlabel('time(sec)', fontsize = 20)
plt.ylabel('BIS', fontsize = 25, rotation = 90)
plt.ylim(0, 100)
plt.title('time-BIS', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.ticklabel_format(style='plain')

# ax.set_xticks(minor_xticks, minor=True)
ax.yaxis.get_ticklocs(minor=True)
ax.minorticks_on()
ax.xaxis.get_ticklocs(minor=True)

ax.tick_params(axis='x', labelsize=20, length=10, width=3)
ax.tick_params(axis='x', which='minor', length=5, width=2)
# plt.fill_between(x, b, color = 'navy')


# # plt.rc('font', size=20)        # 기본 폰트 크기
# plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
# plt.rc('xtick', fontsize=50)  # x축 눈금 폰트 크기
# plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
# plt.rc('legend', fontsize=20)  # 범례 폰트 크기
# plt.rc('figure', titlesize=800) # figure title 폰트 크기

In [ ]:
#time- sqi그래프

x = np.linspace(0,10864,10864)
b = sqi_label

fig, ax = plt.subplots(figsize = (40,4))
plt.plot(x,b,color = 'navy')
plt.xlabel('time(sec)', fontsize = 20)
plt.ylabel('SQI(%)', fontsize = 20)
plt.ylim(0, 100)
plt.title('time-SQI', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.ticklabel_format(style='plain')

ax.yaxis.get_ticklocs(minor=True)
ax.minorticks_on()
ax.xaxis.get_ticklocs(minor=True)

ax.tick_params(axis='x', labelsize=20, length=10, width=3)
ax.tick_params(axis='x', which='minor', length=5, width=2)
# plt.fill_between(x, b, color = 'navy')



In [ ]:
#모든 raw data에 대해 전처리하기
#함수만들기?
#train, target

In [ ]:
#전처리하는 함수 만들기

# 전처리 자동화

In [ ]:
import os

path = f'{proj_path}/data/raw'
file_list = os.listdir(path)
print(file_list)

for file in file_list:
    filepath = path + '/' + file
    print(filepath)

In [ ]:
#전처리 과정 담은 함수

def preprocessing(filename):
  # 전처리 과정 다 넣어두기
  # return: 전처리 된 결과물을 파일로 return (control에서 하기)

  filename = np.load(filepath)

  bis=filename[:,0]
  bis_cond = ~np.isnan(bis)
  idx_arr = np.arange(0, len(bis), 1)
  bis_idx = idx_arr[bis_cond]

  sqi = filename[:,1]
  sqi_cond = ~np.isnan(sqi)
  idx_arr_s = np.arange(0, len(sqi), 1)
  sqi_idx = idx_arr_s[sqi_cond]

  eeg = filename[:, 2]
  start = 55*100
  eeg_start_idx = bis_idx - start
  eeg_start_idx = eeg_start_idx[eeg_start_idx >= 0]

  eeg_segments = []
  bis_label = []
  sqi_label = []

  for start_idx in tqdm(eeg_start_idx):
    end_idx = start_idx + 30 * 100
    eeg_segments += list([eeg[start_idx:end_idx]])
    bis_label += list([bis[start_idx]])
    sqi_label += list([sqi[start_idx]])

  eeg_segments = np.array(eeg_segments)
  bis_label = np.array(bis_label)
  sqi_label = np.array(sqi_label)

  sqi_cond2 = sqi_label > 90

  bis_label = bis_label[sqi_cond2]
  sqi_label = sqi_label[sqi_cond2]
  eeg_selected = eeg_segments[sqi_cond2]

  inputs = eeg_selected
  outputs = bis_label

  return inputs, outputs
  # 튜플


In [ ]:
#전처리 총괄하는 함수

def control():

  path = f'{proj_path}/data/raw'
  input_path =  f'{proj_path}/data/preprocessed/inputs'
  output_path = f'{proj_path}/data/preprocessed/outputs'
  file_list = os.listdir(path)
  print(file_list)

  for file in file_list:
    filepath = path + '/' + file
    print(filepath)
    preprocessing(filepath)
    # preprocessed = preprocessing(filepath)

    inputs, outputs = preprocessing(filepath)

    np.save(input_path + "/" + file + "_input.npy", inputs)
    np.save(output_path + "/" + file + "_output.npy", outputs)

In [ ]:
control()